# Demo - Adversarial Training (MNIST)

In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks
from torchattacks import PGD, FGSM

from models import CNN

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load Data

In [3]:
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

## 2. Define Model

In [5]:
model = CNN().cuda()

In [6]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
pgd_attack = PGD(model, eps=0.3, alpha=0.1, steps=7)

## 3. Train Model

In [8]:
num_epochs = 5

In [9]:
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = pgd_attack(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/5], lter [100/468], Loss: 2.2998
Epoch [1/5], lter [200/468], Loss: 2.0456
Epoch [1/5], lter [300/468], Loss: 1.9646
Epoch [1/5], lter [400/468], Loss: 1.6566
Epoch [2/5], lter [100/468], Loss: 1.3704
Epoch [2/5], lter [200/468], Loss: 1.1036
Epoch [2/5], lter [300/468], Loss: 1.3445
Epoch [2/5], lter [400/468], Loss: 1.0688
Epoch [3/5], lter [100/468], Loss: 0.8610
Epoch [3/5], lter [200/468], Loss: 0.7127
Epoch [3/5], lter [300/468], Loss: 0.8645
Epoch [3/5], lter [400/468], Loss: 0.6896
Epoch [4/5], lter [100/468], Loss: 0.6138
Epoch [4/5], lter [200/468], Loss: 0.5215
Epoch [4/5], lter [300/468], Loss: 0.6302
Epoch [4/5], lter [400/468], Loss: 0.5148
Epoch [5/5], lter [100/468], Loss: 0.5115
Epoch [5/5], lter [200/468], Loss: 0.4259
Epoch [5/5], lter [300/468], Loss: 0.5392
Epoch [5/5], lter [400/468], Loss: 0.3859


## 4. Test Model

### 4.1 Standard Accuracy

In [10]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 96.51 %


### 4.2 Robust Accuracy

In [11]:
model.eval()

correct = 0
total = 0

fgsm_attack = FGSM(model, eps=0.3)

for images, labels in test_loader:
    
    images = fgsm_attack(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 89.76 %
